In [1]:
import matplotlib.pyplot as plt

from dotenv import load_dotenv 
import os 

import pandas as pd
import geopandas as gpd

from datetime import timedelta

import requests
import json

import time

In [2]:
la_bounds_gdf = gpd.read_file("../../Data/Inputs/Boundary_Shapefile/la_mainland.geojson")

In [12]:
la_ds_gdf = gpd.read_file('../../Data/Outputs/la_ds_gdf.geojson')

In [4]:
load_dotenv()

api_key = os.getenv("noaa_key") 
debug = os.getenv("DEBUG", "False") 

In [9]:
headers = {
    "token": api_key
}

In [10]:
session = requests.Session()
session.headers.update({"token": api_key})

In [11]:
def get_response(url, session=session, retries=3, timeout=60):
    for attempt in range(retries):
        try:
            print(f"Making request to {url}")
            response = session.get(url, timeout=timeout)
            response.raise_for_status()
            data = response.json()
            time.sleep(3)
            return data.get('results', [])
        except requests.exceptions.HTTPError as e:
            print(f"HTTP error {e} for URL {url}")
            return None
        except requests.exceptions.RequestException as e:
            print(f"Request error {e}, attempt {attempt + 1}/{retries}")
        except ValueError as e:
            print(f"JSON decode error for URL {url}: {e}")
            return None
    print(f"Failed to get data from {url} after {retries} attempts")
    return None


In [6]:
def to_gdf(results_json):
    results_df = pd.DataFrame(results_json)
    results_gdf = gpd.GeoDataFrame(
   results_df, geometry=gpd.points_from_xy(results_df['longitude'], results_df['latitude']), crs="EPSG:4326"
)
    return results_gdf